In [ ]:
# pip install PyMySQL
# PyMySQL is already installed in the virtual environment

# All the steps from the Example

In [4]:
import pymysql.cursors
import pandas as pd

In [2]:
host = 'den1.mysql6.gear.host'
db = 'situation'
usr = 'situation'
psword = input()

# Connect to the database
connection = pymysql.connect(host=host, user=usr, password=psword, database=db, cursorclass=pymysql.cursors.DictCursor)

In [5]:
UserId = 'SomeDeviceName'
sql = "SELECT timestamp, sensorValue FROM lims_timeseries WHERE Fkey_limsDevice = %s"

# with connection.cursor() as cursor:
#   cursor.execute(sql % UserId)

df_raw = pd.read_sql(sql, connection, params=(UserId,))

/var/folders/4_/pl6t6pd123545nrgljv7gy340000gn/T/ipykernel_44302/1665320423.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_raw = pd.read_sql(sql, connection, params=(UserId,))


In [6]:
df = df_raw[(df_raw['timestamp'] >= '2022-01-14') & (df_raw['timestamp'] <= '2024-02-01')]
df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.set_index('timestamp')
df = df.resample('10s').first()
df = df.interpolate()

In [7]:
df.head(10)

,sensorValue
timestamp,


# Trying to access the database used in B2

In [13]:
from sqlalchemy import create_engine

In [9]:
with connection.cursor() as cursor:
    # get all tables'name
    cursor.execute("SHOW TABLES;")
    tables = cursor.fetchall()
    print(tables)

[{'Tables_in_situation': 'a1_experiments'}, {'Tables_in_situation': 'a1_reacts'}, {'Tables_in_situation': 'a1_results'}, {'Tables_in_situation': 'aspirin_synthesis_workflow'}, {'Tables_in_situation': 'b2_molecule_solubility'}, {'Tables_in_situation': 'b2_molecules'}, {'Tables_in_situation': 'c3_device_properties'}, {'Tables_in_situation': 'c3_devicecalibration'}, {'Tables_in_situation': 'c3_devices'}, {'Tables_in_situation': 'c3_devices_depr'}, {'Tables_in_situation': 'c3_devices_properties_temp'}, {'Tables_in_situation': 'chemicals_inventory'}, {'Tables_in_situation': 'copper_sulfate_pentahydrate_synthesis_workflow'}, {'Tables_in_situation': 'd4_device'}, {'Tables_in_situation': 'd4_les_workflow'}, {'Tables_in_situation': 'd4_timeseriesdata'}, {'Tables_in_situation': 'e5_cdb_analysis'}, {'Tables_in_situation': 'e5_cdb_column'}, {'Tables_in_situation': 'e5_cdb_diagram_data'}, {'Tables_in_situation': 'e5_cdb_method'}, {'Tables_in_situation': 'educts'}, {'Tables_in_situation': 'eln_exper

In [14]:
connection_string = f"mysql+pymysql://{'situation'}:{'cogni88.'}@{'den1.mysql6.gear.host'}/{'situation'}"
engine = create_engine(connection_string)


tables_to_read = ['b2_molecule_solubility', 'b2_molecules']
        
dataframes = {}

for table_name in tables_to_read:
    # using pandas to read the data
    query = f"SELECT * FROM `{table_name}`"
    df = pd.read_sql_query(query, engine)
    dataframes[table_name] = df
    print(f"DataFrame for {table_name}:")
    print(df.head())

DataFrame for b2_molecule_solubility:
   id molecule_name       smiles  solubility
0   1     Molecule1  C1=CC=CC=C1     1.59763
1   2     Molecule2  C2=CC=CC=C2     1.93038
2   3     Molecule3  C3=CC=CC=C3     1.70553
3   4     Molecule4  C4=CC=CC=C4     1.58977
4   5     Molecule5  C5=CC=CC=C5     1.34731
DataFrame for b2_molecules:
   MoleculeID  FkeyUser   MoleculeName           CanonicalSmileFormat  \
0           1         1  Acetaminophen          CC(=O)NC1=CC=C(C=C1)O   
1           2         1        Aspirin       CC(=O)OC1=CC=CC=C1C(=O)O   
2           3         1       Caffeine   CN1C=NC2=C1C(=O)N(C(=O)N2C)C   
3           4         1      Ibuprofen  CC(C)CC1=CC=C(C=C1)C(C)C(=O)O   
4           5         1  Butyl Acetate                    CCCCOC(=O)C   

        CasId   Formular                                            MOLFile  
0    103-90-2    C8H9NO2  1983\n  -OEChem-11082306173D\n\n 20 20  0     ...  
1     50-78-2     C9H8O4  2244\n  -OEChem-11082306153D\n\n 21 21  0  